In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectFromModel

from lifelines import CoxPHFitter
from lifelines.datasets import load_rossi

In [ ]:
df_expression = pd.read_csv(r'c:\...\prad_expMat_full.csv', delimiter=',')
df_operation = pd.read_csv(r'c:\...\org_clinical_omf_v4.0_prad.csv',  delimiter=',')
df_recurrence = pd.read_csv(r'c:\...\org_clinical_follow_up_v1.0_prad.csv',  delimiter=',')

In [3]:
df_expression=df_expression.T

In [4]:
df_expression.head(5)

,0,1,2,3,4,5,6,7,8,9,...,60650,60651,60652,60653,60654,60655,60656,60657,60658,60659
Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
TCGA-HC-A6AP-01A-11R-A30B-07,4158,138,1571,379,140,420,1521,3964,1820,651,...,0,0,9,0,1,0,154,0,3,77
TCGA-KK-A7B2-01A-12R-A32O-07,2637,2,1436,452,123,584,850,1758,1975,857,...,0,0,11,0,0,0,158,0,1,35
TCGA-EJ-A8FO-01A-21R-A36G-07,4429,2,1765,1069,183,250,742,2857,2786,2184,...,0,0,5,0,0,0,337,0,3,17
TCGA-G9-6377-01A-11R-1965-07,4314,5,1454,845,149,279,722,3888,1788,2468,...,0,0,7,0,0,0,292,0,0,24


In [5]:
#Урезаем кодировку всех пациентов до шаблона TCGA-##-####, потому что этого хватает для их идентификации
df_expression=df_expression.rename(index=lambda s: s[:12])

In [6]:
df_expression.head()

,0,1,2,3,4,5,6,7,8,9,...,60650,60651,60652,60653,60654,60655,60656,60657,60658,60659
Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
TCGA-HC-A6AP,4158,138,1571,379,140,420,1521,3964,1820,651,...,0,0,9,0,1,0,154,0,3,77
TCGA-KK-A7B2,2637,2,1436,452,123,584,850,1758,1975,857,...,0,0,11,0,0,0,158,0,1,35
TCGA-EJ-A8FO,4429,2,1765,1069,183,250,742,2857,2786,2184,...,0,0,5,0,0,0,337,0,3,17
TCGA-G9-6377,4314,5,1454,845,149,279,722,3888,1788,2468,...,0,0,7,0,0,0,292,0,0,24


In [7]:
#Делаем индексы строк дополнительным столбцом датафрейма
df_expression = df_expression.reset_index()

In [8]:
df_expression.head()

,index,0,1,2,3,4,5,6,7,8,...,60650,60651,60652,60653,60654,60655,60656,60657,60658,60659
0,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
1,TCGA-HC-A6AP,4158,138,1571,379,140,420,1521,3964,1820,...,0,0,9,0,1,0,154,0,3,77
2,TCGA-KK-A7B2,2637,2,1436,452,123,584,850,1758,1975,...,0,0,11,0,0,0,158,0,1,35
3,TCGA-EJ-A8FO,4429,2,1765,1069,183,250,742,2857,2786,...,0,0,5,0,0,0,337,0,3,17
4,TCGA-G9-6377,4314,5,1454,845,149,279,722,3888,1788,...,0,0,7,0,0,0,292,0,0,24


In [9]:
df_operation.head()

,Unnamed: 0,bcr_patient_uuid,bcr_patient_barcode,bcr_omf_barcode,bcr_omf_uuid,form_completion_date,malignancy_type,other_malignancy_dx_days_to,surgery_indicator,other_malignancy_surgery_type,...,ajcc_nodes_pathologic_pn,ajcc_metastasis_pathologic_pm,ajcc_pathologic_tumor_stage,clinical_stage,other_malignancy_anatomic_site,other_malignancy_anatomic_site_text,other_malignancy_histological_type,other_malignancy_histological_type_text,other_malignancy_laterality,stage_other
0,0,bcr_patient_uuid,bcr_patient_barcode,bcr_omf_barcode,bcr_omf_uuid,form_completion_date,malignancy_type,days_to_other_malignancy_dx,surgery_indicator,surgery_type,...,pathologic_N,pathologic_M,pathologic_stage,clinical_stage,other_malignancy_anatomic_site,other_malignancy_anatomic_site_text,other_malignancy_histological_type,other_malignancy_histological_type_text,other_malignancy_laterality,stage_other
1,1,CDE_ID:,CDE_ID:2003301,CDE_ID:,CDE_ID:,CDE_ID:,CDE_ID:3182890,CDE_ID:3462301,CDE_ID:3186538,CDE_ID:3462300,...,CDE_ID:3203106,CDE_ID:3045439,CDE_ID:3203222,CDE_ID:3440332,CDE_ID:4122351,CDE_ID:4122371,CDE_ID:4122411,CDE_ID:4122431,CDE_ID:4122391,CDE_ID:2007104
2,2,928c48a0-68ee-4e28-ae83-9832e52850ca,TCGA-CH-5753,TCGA-CH-5753-O30033,B2466F9B-13AF-46FD-B6D9-E649C73D181A,2012-3-21,Synchronous Malignancy,[Not Available],[Not Available],[Not Available],...,[Not Available],[Not Available],[Not Available],[Not Applicable],Lymph node(s),[Not Applicable],"Other, specify",Non-Hodgkin's,[Not Available],[Not Available]
3,3,18e79a01-07a2-4cd5-b700-71d41724e041,TCGA-CH-5763,TCGA-CH-5763-O30027,111B40BE-4E53-4D24-AD78-955DCAF14EA7,2012-3-21,Prior Malignancy,[Not Available],[Not Available],[Not Available],...,N0,M0,Stage IA,[Not Applicable],Lung,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available]
4,4,ed73f136-42e8-4100-b432-d10370d07e8b,TCGA-CH-5764,TCGA-CH-5764-O30030,9611057D-FF65-4176-9ED2-A6D3D3749351,2012-3-21,Prior Malignancy,[Not Available],[Not Available],[Not Available],...,N1,M0,Stage III,[Not Applicable],Esophagus,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available]


In [10]:
#Удаляем первые две строки
df_operation = df_operation.drop(labels=[0,1])

In [11]:
#Выделяем айди пациентов с операционным лечением в отдельный датафрейм
df_id_operation_patient = df_operation[['bcr_patient_barcode']].copy()

In [12]:
df_id_operation_patient.head()

,bcr_patient_barcode
2,TCGA-CH-5753
3,TCGA-CH-5763
4,TCGA-CH-5764
5,TCGA-CH-5772
6,TCGA-CH-5790


In [13]:
#Объединяем пересечением таблицу экспрессий генов всех пациентов и тех, у кого было операционное лечение
df_patient_operation=pd.merge(
    df_expression, df_id_operation_patient,
    left_on='index',
    right_on='bcr_patient_barcode',
    how='inner'
)

In [14]:
df_patient_operation.head()

,index,0,1,2,3,4,5,6,7,8,...,60651,60652,60653,60654,60655,60656,60657,60658,60659,bcr_patient_barcode
0,TCGA-J9-A8CN,1904,1,959,390,84,296,302,2311,2207,...,0,11,0,0,0,239,0,2,25,TCGA-J9-A8CN
1,TCGA-EJ-7325,3026,0,894,542,41,70,373,3124,985,...,0,6,0,0,0,108,0,3,35,TCGA-EJ-7325
2,TCGA-KK-A8IB,3964,2,1580,491,322,503,789,2720,2073,...,0,20,0,1,0,193,0,6,10,TCGA-KK-A8IB
3,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,9,0,0,0,299,0,5,21,TCGA-EJ-A65J
4,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,9,0,0,0,299,0,5,21,TCGA-EJ-A65J


In [15]:
#Удаляем столбец-дубликат с кодами пациентов
df_patient_operation=df_patient_operation.drop(columns="bcr_patient_barcode")

In [16]:
#Создаем датафрейм из строки с названиями генов
df_genes_id = df_expression[0:1]
df_genes_id

,index,0,1,2,3,4,5,6,7,8,...,60650,60651,60652,60653,60654,60655,60656,60657,60658,60659
0,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1


In [17]:
#присоединяем строку с генами к таблице
df_patient_operation = pd.concat([df_patient_operation, df_genes_id])

In [18]:
df_patient_operation

,index,0,1,2,3,4,5,6,7,8,...,60650,60651,60652,60653,60654,60655,60656,60657,60658,60659
0,TCGA-J9-A8CN,1904,1,959,390,84,296,302,2311,2207,...,0,0,11,0,0,0,239,0,2,25
1,TCGA-EJ-7325,3026,0,894,542,41,70,373,3124,985,...,0,0,6,0,0,0,108,0,3,35
2,TCGA-KK-A8IB,3964,2,1580,491,322,503,789,2720,2073,...,0,0,20,0,1,0,193,0,6,10
3,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,0,9,0,0,0,299,0,5,21
4,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,0,9,0,0,0,299,0,5,21
5,TCGA-EJ-A46G,4427,4,868,518,230,128,511,2004,817,...,0,0,41,0,0,0,77,0,7,53
6,TCGA-KK-A7B3,2701,2,1495,783,150,897,1212,3222,2019,...,0,0,64,0,1,0,408,0,13,65
7,TCGA-EJ-A6RC,3745,49,1329,897,208,427,2215,2642,2092,...,0,0,11,0,0,0,332,0,9,33
8,TCGA-HC-7233,4011,14,1546,1093,230,368,1608,3542,3051,...,0,0,8,0,0,1,293,0,4,36
9,TCGA-KC-A7FA,2458,92,1286,628,130,296,337,2580,2192,...,0,0,25,0,0,0,207,0,3,37


In [19]:
#Называем столбцы таблицы названиями генов и удаляем строку с названиями генов
df_patient_operation.columns=df_patient_operation.iloc[33]
df_patient_operation=df_patient_operation.loc[df_patient_operation['Unnamed: 0']!='Unnamed: 0']

In [20]:
df_patient_operation

,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
0,TCGA-J9-A8CN,1904,1,959,390,84,296,302,2311,2207,...,0,0,11,0,0,0,239,0,2,25
1,TCGA-EJ-7325,3026,0,894,542,41,70,373,3124,985,...,0,0,6,0,0,0,108,0,3,35
2,TCGA-KK-A8IB,3964,2,1580,491,322,503,789,2720,2073,...,0,0,20,0,1,0,193,0,6,10
3,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,0,9,0,0,0,299,0,5,21
4,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,0,9,0,0,0,299,0,5,21
5,TCGA-EJ-A46G,4427,4,868,518,230,128,511,2004,817,...,0,0,41,0,0,0,77,0,7,53
6,TCGA-KK-A7B3,2701,2,1495,783,150,897,1212,3222,2019,...,0,0,64,0,1,0,408,0,13,65
7,TCGA-EJ-A6RC,3745,49,1329,897,208,427,2215,2642,2092,...,0,0,11,0,0,0,332,0,9,33
8,TCGA-HC-7233,4011,14,1546,1093,230,368,1608,3542,3051,...,0,0,8,0,0,1,293,0,4,36
9,TCGA-KC-A7FA,2458,92,1286,628,130,296,337,2580,2192,...,0,0,25,0,0,0,207,0,3,37


In [21]:
df_bcr_patient_barcode=df_operation[['bcr_patient_barcode']].copy()
df_bcr_patient_barcode=df_bcr_patient_barcode.reset_index(drop=True)
df_bcr_patient_barcode

,bcr_patient_barcode
0,TCGA-CH-5753
1,TCGA-CH-5763
2,TCGA-CH-5764
3,TCGA-CH-5772
4,TCGA-CH-5790
5,TCGA-CH-5791
6,TCGA-EJ-5542
7,TCGA-EJ-7123
8,TCGA-EJ-7312
9,TCGA-EJ-7325


In [22]:
#Создаём датафрейм с днями до хирургии
df_days_to_surgery=df_operation[['other_malignancy_surgery_days_to']].copy()
df_days_to_surgery=df_days_to_surgery.reset_index(drop=True)
df_days_to_surgery

,other_malignancy_surgery_days_to
0,[Not Available]
1,-91
2,-457
3,-4840
4,-2740
5,[Not Available]
6,[Not Available]
7,[Not Available]
8,[Not Available]
9,-119


In [23]:
df_days_to_resection=df_bcr_patient_barcode.join(df_days_to_surgery)
df_days_to_resection

,bcr_patient_barcode,other_malignancy_surgery_days_to
0,TCGA-CH-5753,[Not Available]
1,TCGA-CH-5763,-91
2,TCGA-CH-5764,-457
3,TCGA-CH-5772,-4840
4,TCGA-CH-5790,-2740
5,TCGA-CH-5791,[Not Available]
6,TCGA-EJ-5542,[Not Available]
7,TCGA-EJ-7123,[Not Available]
8,TCGA-EJ-7312,[Not Available]
9,TCGA-EJ-7325,-119


In [24]:
# Объединяем пересечением таблицу экспрессий генов всех пациентов и тех, у кого были дни до операции
df_patient_operation=pd.merge(
    df_patient_operation, df_days_to_resection,
    left_on='Unnamed: 0',
    right_on='bcr_patient_barcode',
    how='inner'
)

df_patient_operation

,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1,bcr_patient_barcode,other_malignancy_surgery_days_to
0,TCGA-J9-A8CN,1904,1,959,390,84,296,302,2311,2207,...,11,0,0,0,239,0,2,25,TCGA-J9-A8CN,339
1,TCGA-EJ-7325,3026,0,894,542,41,70,373,3124,985,...,6,0,0,0,108,0,3,35,TCGA-EJ-7325,-119
2,TCGA-KK-A8IB,3964,2,1580,491,322,503,789,2720,2073,...,20,0,1,0,193,0,6,10,TCGA-KK-A8IB,-2136
3,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,9,0,0,0,299,0,5,21,TCGA-EJ-A65J,-782
4,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,9,0,0,0,299,0,5,21,TCGA-EJ-A65J,-782
5,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,9,0,0,0,299,0,5,21,TCGA-EJ-A65J,-782
6,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,9,0,0,0,299,0,5,21,TCGA-EJ-A65J,-782
7,TCGA-EJ-A46G,4427,4,868,518,230,128,511,2004,817,...,41,0,0,0,77,0,7,53,TCGA-EJ-A46G,[Not Available]
8,TCGA-KK-A7B3,2701,2,1495,783,150,897,1212,3222,2019,...,64,0,1,0,408,0,13,65,TCGA-KK-A7B3,[Not Available]
9,TCGA-EJ-A6RC,3745,49,1329,897,208,427,2215,2642,2092,...,11,0,0,0,332,0,9,33,TCGA-EJ-A6RC,[Not Available]


In [25]:
df_recurrence.head()

,Unnamed: 0,bcr_patient_uuid,bcr_patient_barcode,bcr_followup_barcode,bcr_followup_uuid,form_completion_date,followup_reason,followup_lost_to,radiation_treatment_adjuvant,pharmaceutical_tx_adjuvant,...,new_tumor_event_dx_days_to,new_tumor_event_site,new_tumor_event_site_other,progression_after_hormone_tx,progression_after_hormone_tx_type,days_to_biochemical_recurrence_second,days_to_biochemical_recurrence_third,new_tumor_event_radiation_tx,new_tumor_event_pharmaceutical_tx,treatment_outcome_at_tcga_followup
0,0,bcr_patient_uuid,bcr_patient_barcode,bcr_followup_barcode,bcr_followup_uuid,form_completion_date,followup_case_report_form_submission_reason,lost_follow_up,radiation_therapy,postoperative_rx_tx,...,days_to_new_tumor_event_after_initial_treatment,new_neoplasm_event_occurrence_anatomic_site,new_neoplasm_occurrence_anatomic_site_text,tumor_progression_post_ht,type_ofprogression_of_disease_ht,days_to_second_biochemical_recurrence,days_to_third_biochemical_recurrence,additional_radiation_therapy,additional_pharmaceutical_therapy,followup_treatment_success
1,1,CDE_ID:,CDE_ID:2003301,CDE_ID:,CDE_ID:,CDE_ID:,CDE_ID:3233305,CDE_ID:61333,CDE_ID:2005312,CDE_ID:3397567,...,CDE_ID:3392464,CDE_ID:3108271,CDE_ID:3128033,CDE_ID:3158820,CDE_ID:3354944,CDE_ID:3414617,CDE_ID:3414621,CDE_ID:3427615,CDE_ID:3427616,CDE_ID:3104050
2,2,49197847-CC83-4CE1-8397-D09CEA4C4928,TCGA-2A-A8VL,TCGA-2A-A8VL-F71759,6E1A6CED-30C0-4A4D-A550-82214C73826D,2015-3-31,Scheduled Follow-up Submission,NO,[Not Available],[Not Available],...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Complete Remission/Response
3,3,91C0D161-2B59-4B7A-8C19-6D26DEA31849,TCGA-2A-A8VO,TCGA-2A-A8VO-F71780,3A8226A3-E762-4CD4-A32A-64E485053187,2015-3-31,Scheduled Follow-up Submission,NO,NO,NO,...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Complete Remission/Response
4,4,931B549F-B9F2-4E8D-83ED-FF663671883C,TCGA-2A-A8VT,TCGA-2A-A8VT-F71798,2194A7E8-9DD7-4E55-9EF6-BACD266E1E90,2015-3-31,Scheduled Follow-up Submission,NO,[Not Available],[Not Available],...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Stable Disease


In [26]:
#Убираем две первые строки
df_recurrence=df_recurrence.drop(labels=[0,1])
df_recurrence

,Unnamed: 0,bcr_patient_uuid,bcr_patient_barcode,bcr_followup_barcode,bcr_followup_uuid,form_completion_date,followup_reason,followup_lost_to,radiation_treatment_adjuvant,pharmaceutical_tx_adjuvant,...,new_tumor_event_dx_days_to,new_tumor_event_site,new_tumor_event_site_other,progression_after_hormone_tx,progression_after_hormone_tx_type,days_to_biochemical_recurrence_second,days_to_biochemical_recurrence_third,new_tumor_event_radiation_tx,new_tumor_event_pharmaceutical_tx,treatment_outcome_at_tcga_followup
2,2,49197847-CC83-4CE1-8397-D09CEA4C4928,TCGA-2A-A8VL,TCGA-2A-A8VL-F71759,6E1A6CED-30C0-4A4D-A550-82214C73826D,2015-3-31,Scheduled Follow-up Submission,NO,[Not Available],[Not Available],...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Complete Remission/Response
3,3,91C0D161-2B59-4B7A-8C19-6D26DEA31849,TCGA-2A-A8VO,TCGA-2A-A8VO-F71780,3A8226A3-E762-4CD4-A32A-64E485053187,2015-3-31,Scheduled Follow-up Submission,NO,NO,NO,...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Complete Remission/Response
4,4,931B549F-B9F2-4E8D-83ED-FF663671883C,TCGA-2A-A8VT,TCGA-2A-A8VT-F71798,2194A7E8-9DD7-4E55-9EF6-BACD266E1E90,2015-3-31,Scheduled Follow-up Submission,NO,[Not Available],[Not Available],...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Stable Disease
5,5,75A7AFB5-66D5-47E3-8A8A-3E3A1E749A96,TCGA-2A-A8VV,TCGA-2A-A8VV-F71800,FC901101-A4AB-466A-BBE9-8ADC403ECF47,2015-3-31,Scheduled Follow-up Submission,NO,[Not Available],[Not Available],...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Complete Remission/Response
6,6,942F1788-D977-4AC0-A177-659F9D4CD077,TCGA-2A-A8VX,TCGA-2A-A8VX-F71804,24877403-12D1-40B7-8F7F-67714B8EAC8A,2015-3-31,Scheduled Follow-up Submission,NO,[Not Available],[Not Available],...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Complete Remission/Response
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,505,9B5D36B2-8B46-48A4-AFA7-0E1936F2D315,TCGA-ZG-A9M4,TCGA-ZG-A9M4-F67945,710894EA-9C17-4F77-B5BA-8D202410B07E,2014-11-13,Scheduled Follow-up Submission,NO,NO,NO,...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Stable Disease
506,506,7EFC26B2-EFF9-4F1E-9F4B-F26AF072C84A,TCGA-ZG-A9MC,TCGA-ZG-A9MC-F67960,4A29D298-9543-47AB-9652-3BDCA1E640BE,2014-11-13,Scheduled Follow-up Submission,NO,YES,NO,...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Stable Disease
507,507,305B0B57-5952-43C7-9063-7BCD954EB44D,TCGA-ZG-A9N3,TCGA-ZG-A9N3-F68209,1979FBD3-99F4-4D7E-B4D2-66081FE30E6A,2014-11-14,Scheduled Follow-up Submission,NO,NO,YES,...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Stable Disease
508,508,348C2BF3-2146-4990-9ADF-FA6968B4F072,TCGA-ZG-A9ND,TCGA-ZG-A9ND-F68219,6CB2EFFB-8333-405A-906A-0F0AB3350F5D,2014-11-14,Scheduled Follow-up Submission,NO,NO,NO,...,[Not Applicable],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Stable Disease


In [27]:
#Создаем датафрейм с айди пациентов
df_bcr_patient_barcode=df_recurrence[['bcr_patient_barcode']].copy()
df_bcr_patient_barcode=df_bcr_patient_barcode.reset_index(drop=True)
df_bcr_patient_barcode

,bcr_patient_barcode
0,TCGA-2A-A8VL
1,TCGA-2A-A8VO
2,TCGA-2A-A8VT
3,TCGA-2A-A8VV
4,TCGA-2A-A8VX
...,...
503,TCGA-ZG-A9M4
504,TCGA-ZG-A9MC
505,TCGA-ZG-A9N3
506,TCGA-ZG-A9ND


In [28]:
#Создаем датафрейм со статусом рецидива после первого лечения
df_recurrence_after_first_treatment=df_recurrence[['new_tumor_event_dx_indicator']].copy()
df_recurrence_after_first_treatment=df_recurrence_after_first_treatment.reset_index(drop=True)
df_recurrence_after_first_treatment

,new_tumor_event_dx_indicator
0,NO
1,NO
2,NO
3,NO
4,NO
...,...
503,NO
504,NO
505,NO
506,NO


In [29]:
#Создаем датафрейм с днями до первого новообразования после певроначального лечения
df_day_recurrence_after_first_treatment=df_recurrence[['new_tumor_event_dx_days_to']].copy()
df_day_recurrence_after_first_treatment=df_day_recurrence_after_first_treatment.reset_index(drop=True)
df_day_recurrence_after_first_treatment


,new_tumor_event_dx_days_to
0,[Not Applicable]
1,[Not Applicable]
2,[Not Applicable]
3,[Not Applicable]
4,[Not Applicable]
...,...
503,[Not Applicable]
504,[Not Applicable]
505,[Not Applicable]
506,[Not Applicable]


In [30]:
#Создаем датафрейм с днями до второго биохимического рецидива после лечения
df_day_recurrence_after_second_treatment=df_recurrence[['days_to_biochemical_recurrence_second']].copy()
df_day_recurrence_after_second_treatment=df_day_recurrence_after_second_treatment.reset_index(drop=True)
df_day_recurrence_after_second_treatment


,days_to_biochemical_recurrence_second
0,[Not Available]
1,[Not Available]
2,[Not Available]
3,[Not Available]
4,[Not Available]
...,...
503,[Not Available]
504,[Not Available]
505,[Not Available]
506,[Not Available]


In [31]:
#Создаем датафрейм с днями до третьего биохимического рецидива после лечения
df_day_recurrence_after_third_treatment=df_recurrence[['days_to_biochemical_recurrence_third']].copy()
df_day_recurrence_after_third_treatment=df_day_recurrence_after_third_treatment.reset_index(drop=True)
df_day_recurrence_after_third_treatment

,days_to_biochemical_recurrence_third
0,[Not Available]
1,[Not Available]
2,[Not Available]
3,[Not Available]
4,[Not Available]
...,...
503,[Not Available]
504,[Not Available]
505,[Not Available]
506,[Not Available]


In [32]:
#Соединяем датафреймы
df_recurrences=df_bcr_patient_barcode.join(df_recurrence_after_first_treatment)
df_recurrences=df_recurrences.join(df_day_recurrence_after_first_treatment)
df_recurrences=df_recurrences.join(df_day_recurrence_after_second_treatment)
df_recurrences=df_recurrences.join(df_day_recurrence_after_third_treatment)
df_recurrences

,bcr_patient_barcode,new_tumor_event_dx_indicator,new_tumor_event_dx_days_to,days_to_biochemical_recurrence_second,days_to_biochemical_recurrence_third
0,TCGA-2A-A8VL,NO,[Not Applicable],[Not Available],[Not Available]
1,TCGA-2A-A8VO,NO,[Not Applicable],[Not Available],[Not Available]
2,TCGA-2A-A8VT,NO,[Not Applicable],[Not Available],[Not Available]
3,TCGA-2A-A8VV,NO,[Not Applicable],[Not Available],[Not Available]
4,TCGA-2A-A8VX,NO,[Not Applicable],[Not Available],[Not Available]
...,...,...,...,...,...
503,TCGA-ZG-A9M4,NO,[Not Applicable],[Not Available],[Not Available]
504,TCGA-ZG-A9MC,NO,[Not Applicable],[Not Available],[Not Available]
505,TCGA-ZG-A9N3,NO,[Not Applicable],[Not Available],[Not Available]
506,TCGA-ZG-A9ND,NO,[Not Applicable],[Not Available],[Not Available]


In [33]:
#Объединяем пересечением таблицу экспрессий генов всех пациентов и тех, у кого было операционное лечение
df_patient_operation_with_dates=pd.merge(
    df_patient_operation, df_recurrences,
    left_on='Unnamed: 0',
    right_on='bcr_patient_barcode',
    how='inner'
)
df_patient_operation_with_dates

,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1,bcr_patient_barcode_x,other_malignancy_surgery_days_to,bcr_patient_barcode_y,new_tumor_event_dx_indicator,new_tumor_event_dx_days_to,days_to_biochemical_recurrence_second,days_to_biochemical_recurrence_third
0,TCGA-EJ-7325,3026,0,894,542,41,70,373,3124,985,...,0,3,35,TCGA-EJ-7325,-119,TCGA-EJ-7325,NO,[Not Applicable],[Not Available],[Not Available]
1,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,5,21,TCGA-EJ-A65J,-782,TCGA-EJ-A65J,NO,[Not Applicable],[Not Available],[Not Available]
2,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,5,21,TCGA-EJ-A65J,-782,TCGA-EJ-A65J,NO,[Not Applicable],[Not Available],[Not Available]
3,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,5,21,TCGA-EJ-A65J,-782,TCGA-EJ-A65J,NO,[Not Applicable],[Not Available],[Not Available]
4,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,5,21,TCGA-EJ-A65J,-782,TCGA-EJ-A65J,NO,[Not Applicable],[Not Available],[Not Available]
5,TCGA-EJ-A46G,4427,4,868,518,230,128,511,2004,817,...,0,7,53,TCGA-EJ-A46G,[Not Available],TCGA-EJ-A46G,NO,[Not Applicable],[Not Available],[Not Available]
6,TCGA-KK-A7B3,2701,2,1495,783,150,897,1212,3222,2019,...,0,13,65,TCGA-KK-A7B3,[Not Available],TCGA-KK-A7B3,YES,294,[Not Available],[Not Available]
7,TCGA-EJ-A6RC,3745,49,1329,897,208,427,2215,2642,2092,...,0,9,33,TCGA-EJ-A6RC,[Not Available],TCGA-EJ-A6RC,NO,[Not Applicable],[Not Available],[Not Available]
8,TCGA-HC-7233,4011,14,1546,1093,230,368,1608,3542,3051,...,0,4,36,TCGA-HC-7233,[Not Available],TCGA-HC-7233,NO,[Not Applicable],[Not Available],[Not Available]
9,TCGA-QU-A6IM,1595,5,453,185,62,172,280,1951,861,...,0,0,60,TCGA-QU-A6IM,[Not Available],TCGA-QU-A6IM,NO,[Not Applicable],[Not Available],[Not Available]


In [34]:
#Удаляем повторяющийся столбец с айди пациента
df_patient_operation_with_dates=df_patient_operation_with_dates.drop(columns='bcr_patient_barcode_x')
df_patient_operation_with_dates=df_patient_operation_with_dates.drop(columns='bcr_patient_barcode_y')

In [35]:
#Итоговая таблица
df_patient_operation_with_dates

,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1,other_malignancy_surgery_days_to,new_tumor_event_dx_indicator,new_tumor_event_dx_days_to,days_to_biochemical_recurrence_second,days_to_biochemical_recurrence_third
0,TCGA-EJ-7325,3026,0,894,542,41,70,373,3124,985,...,0,108,0,3,35,-119,NO,[Not Applicable],[Not Available],[Not Available]
1,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,299,0,5,21,-782,NO,[Not Applicable],[Not Available],[Not Available]
2,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,299,0,5,21,-782,NO,[Not Applicable],[Not Available],[Not Available]
3,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,299,0,5,21,-782,NO,[Not Applicable],[Not Available],[Not Available]
4,TCGA-EJ-A65J,3089,1,1836,847,161,234,473,4509,2808,...,0,299,0,5,21,-782,NO,[Not Applicable],[Not Available],[Not Available]
5,TCGA-EJ-A46G,4427,4,868,518,230,128,511,2004,817,...,0,77,0,7,53,[Not Available],NO,[Not Applicable],[Not Available],[Not Available]
6,TCGA-KK-A7B3,2701,2,1495,783,150,897,1212,3222,2019,...,0,408,0,13,65,[Not Available],YES,294,[Not Available],[Not Available]
7,TCGA-EJ-A6RC,3745,49,1329,897,208,427,2215,2642,2092,...,0,332,0,9,33,[Not Available],NO,[Not Applicable],[Not Available],[Not Available]
8,TCGA-HC-7233,4011,14,1546,1093,230,368,1608,3542,3051,...,1,293,0,4,36,[Not Available],NO,[Not Applicable],[Not Available],[Not Available]
9,TCGA-QU-A6IM,1595,5,453,185,62,172,280,1951,861,...,0,125,0,0,60,[Not Available],NO,[Not Applicable],[Not Available],[Not Available]


In [36]:
df_patient_operation_with_dates.drop([10, 11], inplace=True)

In [37]:
# 0 - после операционного лечения не было рецидива, 1 - после оперционного лечения был рецидив
df_patient_operation_with_dates=df_patient_operation_with_dates.assign(status=[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0])

In [38]:
df_patient_operation_with_dates=df_patient_operation_with_dates.drop(columns=['other_malignancy_surgery_days_to', 'new_tumor_event_dx_indicator', 'new_tumor_event_dx_days_to', 'days_to_biochemical_recurrence_second', 'days_to_biochemical_recurrence_third'])


In [39]:
df_patient_operation_with_dates=df_patient_operation_with_dates.drop(columns=['Unnamed: 0'])


In [ ]:
df_patient_operation_with_dates

,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1,status
0,3026,0,894,542,41,70,373,3124,985,567,...,0,6,0,0,0,108,0,3,35,0
1,3089,1,1836,847,161,234,473,4509,2808,2105,...,0,9,0,0,0,299,0,5,21,0
2,3089,1,1836,847,161,234,473,4509,2808,2105,...,0,9,0,0,0,299,0,5,21,0
3,3089,1,1836,847,161,234,473,4509,2808,2105,...,0,9,0,0,0,299,0,5,21,0
4,3089,1,1836,847,161,234,473,4509,2808,2105,...,0,9,0,0,0,299,0,5,21,0
5,4427,4,868,518,230,128,511,2004,817,2072,...,0,41,0,0,0,77,0,7,53,0
6,2701,2,1495,783,150,897,1212,3222,2019,1716,...,0,64,0,1,0,408,0,13,65,1
7,3745,49,1329,897,208,427,2215,2642,2092,1205,...,0,11,0,0,0,332,0,9,33,0
8,4011,14,1546,1093,230,368,1608,3542,3051,2261,...,0,8,0,0,1,293,0,4,36,0
9,1595,5,453,185,62,172,280,1951,861,645,...,0,0,0,0,0,125,0,0,60,0


In [ ]:
# Сохранение таблицы
df_patient_operation_with_dates.to_csv(r"c:\...\df_patient_operation_with_status.csv")